In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

import nltk
from utils import make_dataset, preprocess_txt, create_corpus, create_vocab, show_most_freq_n

%reload_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [37]:
# read all datasets
df_appceleratorstudio = pd.read_csv('./data/appceleratorstudio.csv')
df_aptanastudio = pd.read_csv('./data/aptanastudio.csv')
df_bamboo = pd.read_csv('./data/bamboo.csv')
df_clover = pd.read_csv('./data/clover.csv')
df_datamanagement = pd.read_csv('./data/datamanagement.csv')
df_duracloud = pd.read_csv('./data/duracloud.csv')
df_jirasoftware = pd.read_csv('./data/jirasoftware.csv')
df_mesos = pd.read_csv('./data/mesos.csv')
df_titanium = pd.read_csv('./data/titanium.csv')
df_usergrid = pd.read_csv('./data/usergrid.csv')

In [5]:
dataset_arr = [df_appceleratorstudio,
        df_aptanastudio,
        df_bamboo,
        df_clover,
        df_datamanagement,
        df_duracloud,
        df_jirasoftware,
        df_mesos,
        df_titanium,
        df_usergrid]

In [6]:
#get union of datasets
text, labels = make_dataset(dataset_arr)

#create corpus
corpus, labels = create_corpus(text, labels)
len(corpus), len(labels)

(12599, 12599)

In [7]:
#create vocabulary
vocab = create_vocab(corpus)
len(vocab)

16613

In [12]:
df_hsc = pd.read_csv('./data/HSC_unl_raw.csv')
df_hsc.head()

,600954,TLE,Add new tag HIL_TAG_ECM_ENI_BUS_STATE,"<p>Until mid August</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>Define tag ID</p>\r\n\r\n<p>#define HIL_TAG_ECM_ENI_BUS_STATE &nbsp;&lt;tbd&gt;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>typedef struct</p>\r\n\r\n<p>{</p>\r\n\r\n<p>&nbsp; /* <span style=""background:yellow;"">Target bus state for ENI files on ChannelInit</span> */</p>\r\n\r\n<p>&nbsp; uint32_t ulTargetBusState;</p>\r\n\r\n<p>} HIL_TAG_ECM_ENI_BUS_STATE_DATA_T;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>typedef struct</p>\r\n\r\n<p>{</p>\r\n\r\n<p>&nbsp; HIL_TAG_HEADER_T&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; tHeader;</p>\r\n\r\n<p>&nbsp; HIL_TAG_ECM_ENI_BUS_STATE_DATA_T tData;</p>\r\n\r\n<p>} HIL_TAG_ECM_ENI_BUS_STATE_T;</p>\r\n\r\n<p>&nbsp;</p>\r\n\r\n<p>#define HIL_TAG_ECM_ENI_BUS_STATE_<span style=""background:yellow;"">OFF 0</span></p>\r\n\r\n<p>#define HIL_TAG_ECM_ENI_BUS_STATE_<span style=""background:yellow;"">ON 1</span></p>\r\n\r\n<p>&nbsp;</p>\r\n",Unnamed: 4
0,546102,TLE,"Win 10 after installation, there is only an un...","<p>Win 10 after installation, there is only an...",NaN
1,546005,TLE,Correct program version in the manual,<p>The manual currently refers to V1.2 of the ...,NaN
2,546003,TLE,Correct program menu link to manual,<p>Correct the file name of the manual (PDF) i...,NaN
3,524365,TLE,Desktop shortcut is installed for admin user,<p>If you install the program as a standard us...,NaN
4,524364,TLE,Number input fields may allow you to input val...,<p>Due to the input/input checking method used...,NaN


In [38]:
df_appceleratorstudio.head()

,issuekey,title,description,storypoint
0,TISTUD-6,Add CA against object literals in function inv...,{html}<div><p>The idea here is that if our met...,1
1,TISTUD-9,Update branding for Appcelerator plugin to App...,"{html}<div><p>At least fix feature icons, asso...",1
2,TISTUD-11,Create new JSON schema for SDK team,{html}<div><p>Create JSON schema containing pr...,1
3,TISTUD-13,Create Project References Property Page,{html}<div><p>Create property page for project...,1
4,TISTUD-16,New Desktop Project Wizard,{html}<div><p>Desktop (need to convert existin...,1


Goal is to change df_appceleratorstudio and other datasets to be like df_hsc.

In [39]:
# split 'issuekey' to to columns
df_appceleratorstudio[['issuekey', 'number']] = df_appceleratorstudio['issuekey'].str.split('-', expand=True)
df_appceleratorstudio.head()

,issuekey,title,description,storypoint,number
0,TISTUD,Add CA against object literals in function inv...,{html}<div><p>The idea here is that if our met...,1,6
1,TISTUD,Update branding for Appcelerator plugin to App...,"{html}<div><p>At least fix feature icons, asso...",1,9
2,TISTUD,Create new JSON schema for SDK team,{html}<div><p>Create JSON schema containing pr...,1,11
3,TISTUD,Create Project References Property Page,{html}<div><p>Create property page for project...,1,13
4,TISTUD,New Desktop Project Wizard,{html}<div><p>Desktop (need to convert existin...,1,16


In [41]:
# change column order of 'number'
df_appceleratorstudio = df_appceleratorstudio[['number', 'issuekey', 'title', 'description', 'storypoint']]
df_appceleratorstudio.head()

,number,issuekey,title,description,storypoint
0,6,TISTUD,Add CA against object literals in function inv...,{html}<div><p>The idea here is that if our met...,1
1,9,TISTUD,Update branding for Appcelerator plugin to App...,"{html}<div><p>At least fix feature icons, asso...",1
2,11,TISTUD,Create new JSON schema for SDK team,{html}<div><p>Create JSON schema containing pr...,1
3,13,TISTUD,Create Project References Property Page,{html}<div><p>Create property page for project...,1
4,16,TISTUD,New Desktop Project Wizard,{html}<div><p>Desktop (need to convert existin...,1


In [51]:
# make valuse of 'storypoint' column all 1, choose exp. that is not valid timespent
df_appceleratorstudio['storypoint'] = 1
df_appceleratorstudio.head()

,number,issuekey,title,description,storypoint
0,6,TISTUD,Add CA against object literals in function inv...,{html}<div><p>The idea here is that if our met...,1
1,9,TISTUD,Update branding for Appcelerator plugin to App...,"{html}<div><p>At least fix feature icons, asso...",1
2,11,TISTUD,Create new JSON schema for SDK team,{html}<div><p>Create JSON schema containing pr...,1
3,13,TISTUD,Create Project References Property Page,{html}<div><p>Create property page for project...,1
4,16,TISTUD,New Desktop Project Wizard,{html}<div><p>Desktop (need to convert existin...,1


In [52]:
# save without header
df_appceleratorstudio.to_csv('./data/formated/appceleratorstudio_unl_raw.csv', header=False)

In [77]:
def format_ds(ds_name):
    #read dataset
    df = pd.read_csv('./data/' + ds_name + '.csv')
    # split 'issuekey' to to columns
    df[['issuekey', 'number']] = df['issuekey'].str.split('-', expand=True)
    # change column order of 'number'
    df = df[['number', 'issuekey', 'title', 'description', 'storypoint']]
    # make valuse of 'storypoint' column all 1, choose one that is not a valid timespent
    df['storypoint'] = 1
    # make folder
    folder_path = './data/formated/' + ds_name
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    # save without header
    df.to_csv(folder_path + '/' + ds_name + '_unl_raw.csv', header=False)

In [78]:
dataset_names = ['appceleratorstudio', 'aptanastudio', 'bamboo', 'clover', 
                 'datamanagement', 'duracloud', 'jirasoftware', 'mesos', 'titanium', 'usergrid']

In [79]:
for ds_name in dataset_names:
    format_ds(ds_name)